In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from ipywidgets import widgets, VBox, HBox, Layout
from IPython.display import display, clear_output
from Genres import fetch_year_summary_and_plot
from Artist import get_artist_data, plot_data, connect_to_db, validate_artist, split_and_aggregate, get_genre_average
from Top5 import get_artist_data as get_top5_artist_data, validate_years, calculate_rank_value, get_top_artists, display_table, plot_data as plot_top5_data, display_table as display_top5_table

# Main container to display widgets dynamically
output_container = widgets.Output()

# Function to highlight only the 'avg_popularity_artist' cell where it is greater than 'avg_popularity_overall'
def highlight_cells_green(df):
    if 'avg_popularity_artist' in df.columns and 'avg_popularity_overall' in df.columns:
        mask = df['avg_popularity_artist'] > df['avg_popularity_overall']
        
        def apply_green(row):
            styles = ['' for _ in row]
            if mask[row.name]:
                styles[df.columns.get_loc('avg_popularity_artist')] = 'background-color: green'
            return styles
        
        df_styled = df.style.apply(apply_green, axis=1)
        return df_styled
    else:
        print("Columns 'avg_popularity_artist' or 'avg_popularity_overall' not found!")
        return df.style  # Return the original style if columns are missing
    
def style_table_for_top5(df):
    """Style table for Top 5 Artists: Highlight the top artist for each year."""
    df_styled = df.style.apply(highlight_top_artist, axis=1)  # Use highlight_top_artist instead
    df_styled = df_styled.set_properties(**{
        'color': 'black',
        'border-color': 'black',
        'text-align': 'center'
    })
    
    # Apply styling for header cells
    df_styled = df_styled.set_table_styles([{
        'selector': 'th',
        'props': [('background-color', 'lightgray'), ('font-weight', 'bold')]
    }], axis=1)
    
    return df_styled

# Function to highlight the highest popularity in each row (top artist for each year)
# Function to highlight the top artist for each year in red
def highlight_top_artist_red(row):
    max_val = row[row != "Null"].max() if not row.isnull().all() else None
    # Highlight max value with red background
    styles = ['background-color: lightcoral' if v == max_val else '' for v in row]
    return styles



# Updated display_table for Top 5 Artists
def display_table(data):
    """Display the results in a tabular format using Pandas."""
    pivot_data = data.pivot_table(index='ArtistName', columns='Year', values='avg_popularity', aggfunc='mean')
    pivot_data = pivot_data.where(pd.notnull(pivot_data), 'Null')  # Replace NaN with 'Null' for readability

    # Apply red highlighting for top artists
    styled_table = pivot_data.style.apply(highlight_top_artist_red, axis=1)
    return styled_table


# Function to style the pandas DataFrame for better readability in the Top 5 Artists table

def style_table(df):
    # Apply green highlighting for 'avg_popularity_artist' and 'avg_popularity_overall' if needed
    df_styled = highlight_cells_green(df)  # Apply the green highlighting
    df_styled = df_styled.set_properties(**{
        'color': 'black',
        'border-color': 'black',
        'text-align': 'center'
    })
    
    # Apply styling for header cells
    df_styled = df_styled.set_table_styles([{
        'selector': 'th',
        'props': [('background-color', 'lightgray'), ('font-weight', 'bold')]
    }], axis=1)
    
    return df_styled
    

# Function for the Genre interaction
def genre_interaction(year):
    year_summary = fetch_year_summary_and_plot(year)
    
    display(year_summary)

# Function for the Artist interaction
def artist_interaction(artist_name):
    conn = connect_to_db('CWDatabase.db')
    artist_data = get_artist_data(conn, artist_name)
    genre_avg = get_genre_average(conn)
    
    styled_artist_data = style_table(artist_data)
    display(styled_artist_data)
    
    plot_data(artist_data, genre_avg)
    conn.close()

# Function for Top 5 Artists interaction
def top5_interaction(start_year, end_year):
    with output_container:
        clear_output()
        if validate_years(start_year, end_year):
            conn = connect_to_db('CWDatabase.db')
            artist_data = get_top5_artist_data(conn, start_year, end_year)
            ranked_data = calculate_rank_value(artist_data)
            top_artists_data = get_top_artists(ranked_data)
            top_artists_data_one = display_table(top_artists_data)  # Use the modified display_table
            
            if top_artists_data_one is not None:
                display(top_artists_data_one)
            else:
                print("No data to display.")
            
            plot_top5_data(top_artists_data)
            conn.close()
        else:
            print("Invalid input. Please enter valid years between 1998 and 2020.")


# Function to clear the output and show specific widgets
def show_genre_widgets(_):
    with output_container:
        clear_output()
        description_label = widgets.HTML(
            value="<p><strong>🎧 Explore the Popularity by Genre!</strong> Choose a year and press 'Fetch Year Summary' to see genre trends.</p>")
        display(description_label)
        year_slider = widgets.IntSlider(min=1998, max=2020, step=1, description='Year:')
        fetch_button = widgets.Button(description="Fetch Year Summary", style={'button_color': '#D6006B'})
        
        def on_fetch_clicked(b):
            genre_interaction(year_slider.value)
        
        fetch_button.on_click(on_fetch_clicked)
        display(VBox([year_slider, fetch_button]))

# Function to display Artist prompt with emoji
def show_artist_widgets(_):
    with output_container:
        clear_output()
        description_label = widgets.HTML(
            value="<p><strong>🎤 Artist Data</strong> Enter the artist's name to see their performance over the years!</p>")
        display(description_label)
        artist_text = widgets.Text(description='Artist Name:', style={'description_width': 'initial'})
        fetch_button = widgets.Button(description="Fetch Artist Data", style={'button_color': '#D6006B'})
        
        def on_fetch_clicked(b):
            artist_interaction(artist_text.value)
        
        fetch_button.on_click(on_fetch_clicked)
        display(VBox([artist_text, fetch_button]))

# Function to display Top 5 Artists prompt with emoji
def show_top5_widgets(_):
    with output_container:
        clear_output()
        description_label = widgets.HTML(
            value="<p><strong>🏆 Top 5 Artists</strong> Choose a range of years to compare the top artists' popularity!</p>")
        display(description_label)
        start_year_slider = widgets.IntSlider(min=1998, max=2020, step=1, description='Start Year:')
        end_year_slider = widgets.IntSlider(min=1998, max=2020, step=1, description='End Year:')
        fetch_button = widgets.Button(description="Fetch Top 5 Artists", style={'button_color': '#D6006B'})
        
        def on_fetch_clicked(b):
            top5_interaction(start_year_slider.value, end_year_slider.value)
        
        fetch_button.on_click(on_fetch_clicked)
        display(VBox([start_year_slider, end_year_slider, fetch_button]))

# Function to display the description of the 3 buttons
def show_description(_):
    with output_container:
        clear_output()
        description = """
        1. ✅**Genre**: Explore and visualize the popularity of songs by genre over the years.<br>
        2. ✅**Artist**: Get data on a specific artist’s performance and visualize their trends.<br>
        3. ✅**Top 5 Artists**: View and compare the top 5 artists' popularity over a specified range of years.
        """
        description_label = widgets.HTML(value=f'<p style="font-size: 16px; color: #333;">{description}</p>')
        display(description_label)

# New button to show descriptions
description_button = widgets.Button(description="Description", layout=Layout(width='200px', height='50px'), style={'button_color': '#D6006B', 'font_weight': 'bold', 'font_size': '16px'})
description_button.on_click(show_description)

# Main buttons for each category with pink color
genre_button = widgets.Button(description="Genre", layout=Layout(width='200px', height='50px'), style={'button_color': '#D6006B', 'font_weight': 'bold'})
artist_button = widgets.Button(description="Artist", layout=Layout(width='200px', height='50px'), style={'button_color': '#D6006B', 'font_weight': 'bold'})
top5_button = widgets.Button(description="Top 5 Artists", layout=Layout(width='200px', height='50px'), style={'button_color': '#D6006B', 'font_weight': 'bold'})

# Attach event handlers to main buttons
genre_button.on_click(show_genre_widgets)
artist_button.on_click(show_artist_widgets)
top5_button.on_click(show_top5_widgets)

# Buttons layout arranged vertically
buttons_layout = VBox([genre_button, artist_button, top5_button, description_button], layout=Layout(
    justify_content='center',
    align_items='center',
    padding='20px'
))

# Main layout with a large border around the content
header = widgets.HTML('<h1 style="text-align: center; font-size: 36px; color: #D6006B; text-decoration: underline;">🎵 SONG DATA</h1>')

# Updated main layout to include the new button
content_layout = VBox([header, buttons_layout], layout=Layout(
    width='80%',
    padding='20px',
    border='5px solid #D6006B',
    background_color='white',
    margin='20px auto',
    border_radius='12px'
))

# Display the updated layout
display(content_layout, output_container)
